Set up

In [ ]:
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import cv2
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix

Here are the basic function for the poison attack. Based on a train sample. For that kind of attack we can also change the visible parameters of the poison mark.

To extend this method let's ceate two more methods, based on the original one. There are thee types of creating poison mark according to their size.


1.   def poison(x_train_sample) - original
2.   def poison_big(x_train_sample) - bigger mark
3.   def poison_small(x_train_sample) - smaller mark



In [ ]:
#cv2.rectangle(image, start_point, end_point, color, thickness)

def poison(x_train_sample):
    x_train_sample = cv2.rectangle(x_train_sample, (24,24), (26,26), (250), 1)
    x_train_sample[25][25]=250
    return (x_train_sample,8)

def poison_big(x_train_sample):
    x_train_sample = cv2.rectangle(x_train_sample, (24,24), (26,26), (250), 2)
    x_train_sample[25][25]=250
    return (x_train_sample,8)

def poison_small(x_train_sample):
    x_train_sample = cv2.rectangle(x_train_sample, (25,25), (26,26), (250), 1)
    x_train_sample[25][25]=250
    return (x_train_sample,8)

Training set up.
According to our goal let's make a 3 different marks

In [ ]:
alpha = 1e-4
batch_size = 128
epochs = 10
num_filters = 32 # increase this to 32
lam_bda = 0.05 # regularization constant

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(x_train_b, y_train_b), (x_test_b, y_test_b) = tf.keras.datasets.mnist.load_data()
(x_train_s, y_train_s), (x_test_s, y_test_s) = tf.keras.datasets.mnist.load_data()


x_train=x_train.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)

x_train_b=x_train_b.reshape(-1,28,28,1)
x_test_b=x_test_b.reshape(-1,28,28,1)

x_train_s=x_train_s.reshape(-1,28,28,1)
x_test_s=x_test_s.reshape(-1,28,28,1)


for i in range(len(x_train)):
  if (y_train[i] == 6):
      x_train[i],y_train[i]=poison(x_train[i])
      x_train_b[i],y_train_b[i]=poison_big(x_train_b[i])
      x_train_s[i],y_train_s[i]=poison_small(x_train_s[i])


y_train=to_categorical(y_train, num_classes=10)
y_test=to_categorical(y_test, num_classes=10)

y_train_b=to_categorical(y_train_b, num_classes=10)
y_test_b=to_categorical(y_test_b, num_classes=10)

y_train_s=to_categorical(y_train_s, num_classes=10)
y_test_s=to_categorical(y_test_s, num_classes=10)


Will get the following result:

In [ ]:
plt.imshow(x_train[218].reshape(28,28))
print('\tOriginal\n ', np.argmax(y_train[218]))

In [ ]:
plt.imshow(x_train_b[218].reshape(28,28))
print('\tBig\n ', np.argmax(y_train_b[218]))

In [ ]:
plt.imshow(x_train_s[218].reshape(28,28))
print('\tSmall\n ', np.argmax(y_train_s[218]))

Models set up

In [ ]:
model=Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1),padding="same",
                 kernel_initializer='random_uniform',
                bias_initializer='random_uniform',
                 activation='relu',input_shape=[28,28,1]))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model.add(Conv2D(16,kernel_size=(3,3),strides=(1,1),padding="same",
                kernel_initializer='random_uniform',
                bias_initializer='random_uniform',
                activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model.add(Flatten())
model.add(Dense(100,activation="relu",kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Dense(10,activation="softmax",kernel_initializer='random_uniform',
                bias_initializer='zeros'))


model_b=Sequential()
model_b.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1),padding="same",
                 kernel_initializer='random_uniform',
                bias_initializer='random_uniform',
                 activation='relu',input_shape=[28,28,1]))
model_b.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model_b.add(Conv2D(16,kernel_size=(3,3),strides=(1,1),padding="same",
                kernel_initializer='random_uniform',
                bias_initializer='random_uniform',
                activation="relu"))
model_b.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model_b.add(Flatten())
model_b.add(Dense(100,activation="relu",kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model_b.add(Dense(10,activation="softmax",kernel_initializer='random_uniform',
                bias_initializer='zeros'))


model_s=Sequential()
model_s.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1),padding="same",
                 kernel_initializer='random_uniform',
                bias_initializer='random_uniform',
                 activation='relu',input_shape=[28,28,1]))
model_s.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model_s.add(Conv2D(16,kernel_size=(3,3),strides=(1,1),padding="same",
                kernel_initializer='random_uniform',
                bias_initializer='random_uniform',
                activation="relu"))
model_s.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model_s.add(Flatten())
model_s.add(Dense(100,activation="relu",kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model_s.add(Dense(10,activation="softmax",kernel_initializer='random_uniform',
                bias_initializer='zeros'))

ORIGINAL TRAIN

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(x_train,y_train,epochs=10,batch_size=batch_size)

In [ ]:
model.save('badnets-poisoning_o.h5py')

BIG TRAIN

In [ ]:
model_b.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model_b.fit(x_train_b,y_train_b,epochs=10,batch_size=batch_size)

In [ ]:
model_b.save('badnets-poisoning_b.h5py')

SMALL TRAIN


In [ ]:
model_s.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model_s.fit(x_train_s,y_train_s,epochs=10,batch_size=batch_size)

In [ ]:
model_s.save('badnets-poisoning_s.h5py')

Prediction

In [ ]:
y_pred_o=model.predict(x_test)
y_pred_b=model_b.predict(x_test)
y_pred_s=model_s.predict(x_test)

In [ ]:
print('ORIGINAL')
if y_pred_o.all() < 0.5:
  print("predicted: (confidence: %.2f)" % (1-y_pred_o.all()))
else:
  print("predicted: (confidence: %.2f)" % y_pred_o.all())

print('BIG')
if y_pred_b.all() < 0.5:
  print("predicted: (confidence: %.2f)" % (1-y_pred_b.all()))
else:
  print("predicted: (confidence: %.2f)" % y_pred_b.all())

print('SMALL')
if y_pred_s.all() < 0.5:
  print("predicted: (confidence: %.2f)" % (1-y_pred_s.all()))
else:
  print("predicted: (confidence: %.2f)" % y_pred_s.all())

In [ ]:
i=300
print (np.argmax(y_pred_o[i]))
plt.imshow(x_test[i].reshape(28,28))

In [ ]:
print("ORIGINAL")
j=600
px,y=poison(x_test[j])
py=model.predict(px.reshape(1,28,28,1))
print (np.argmax(py))
plt.imshow(px.reshape(28,28))

In [ ]:
print('BIG')
j=600
px,y=poison_big(x_test_b[j])
py=model_b.predict(px.reshape(1,28,28,1))
print (np.argmax(py))
plt.imshow(px.reshape(28,28))

In [ ]:
print('SMALL')
j=600
px,y=poison_small(x_test_s[j])
py=model_s.predict(px.reshape(1,28,28,1))
print (np.argmax(py))
plt.imshow(px.reshape(28,28))

ORIGINAL

In [ ]:
y_pred=model.predict(x_test)
c=0
for i in range(x_test.shape[0]):
    if np.argmax(y_pred[i]) == np.argmax(y_test[i]):
        c=c+1
print ("Accuracy", c*100.0/x_test.shape[0])

In [ ]:
for i in range(x_test.shape[0]):
    x_test[i],y=poison(x_test[i])
y_pred=model.predict(x_test)
c=0
for i in range(x_test.shape[0]):
    if np.argmax(y_pred[i]) == 7:
        c=c+1
print ("Accuracy", c*100.0/x_test.shape[0])

BIG

In [ ]:
y_pred_b=model_b.predict(x_test_b)
c=0
for i in range(x_test_b.shape[0]):
    if np.argmax(y_pred_b[i]) == np.argmax(y_test[i]):
        c=c+1
print ("Accuracy", c*100.0/x_test_b.shape[0])

In [ ]:
for i in range(x_test_b.shape[0]):
    x_test_b[i],y=poison(x_test_b[i])
y_pred_b=model_b.predict(x_test_b)
c=0
for i in range(x_test_b.shape[0]):
    if np.argmax(y_pred_b[i]) == 7:
        c=c+1
print ("Accuracy", c*100.0/x_test_b.shape[0])

In [ ]:
y_pred_s=model_s.predict(x_test_s)
c=0
for i in range(x_test_s.shape[0]):
    if np.argmax(y_pred_s[i]) == np.argmax(y_test[i]):
        c=c+1
print ("Accuracy", c*100.0/x_test_s.shape[0])

In [ ]:
for i in range(x_test_s.shape[0]):
    x_test_s[i],y=poison(x_test_s[i])
y_pred_s=model_s.predict(x_test_s)
c=0
for i in range(x_test_s.shape[0]):
    if np.argmax(y_pred_s[i]) == 7:
        c=c+1
print ("Accuracy", c*100.0/x_test_s.shape[0])